In [63]:
import numpy as np

In [64]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits


In [65]:
import pandas as pd

In [66]:
DATASET = "D:\Aayush\faqs.xlsx"

In [67]:
df=pd.read_excel(DATASET)
df

OSError: [Errno 22] Invalid argument: 'D:\\Aayush\x0caqs.xlsx'

In [9]:
pwd


'C:\\Users\\Admin'

In [2]:
!pip install pandas


In [10]:
cd D:\Aayush

D:\Aayush


In [11]:
pwd

'D:\\Aayush'

In [98]:
import numpy as np

In [99]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits

In [100]:
import pandas as pd

In [101]:
DATASET = "faqs.xlsx"

In [102]:
df=pd.read_excel(DATASET)
df

,Q,A
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [103]:

df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation)
df

,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [104]:
!pip show sentence-transformers


Name: sentence-transformers
Version: 2.1.0
Summary: Sentence Embeddings using BERT / RoBERTa / XLM-R
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: d:\anaconda\lib\site-packages
Requires: scikit-learn, sentencepiece, torchvision, huggingface-hub, nltk, transformers, scipy, torch, tokenizers, numpy, tqdm
Required-by: 


In [105]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [106]:
q_embs = model.encode(df['clean_Q'], convert_to_tensor=True)

D:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [107]:
q_embs.shape

torch.Size([6, 384])

In [108]:
usr_query = "tell me about kandi"

In [109]:
df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
df_usr_query

,usr_query
0,tell me about kandi


In [110]:
df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
df_usr_query

,usr_query,clean_usr_Q
0,tell me about kandi,tell me about kandi


In [111]:

usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
usr_q_emb.shape

D:\Anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


torch.Size([1, 384])

In [112]:
import torch

In [113]:
cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)

In [114]:
idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()

In [115]:
df['A'][idx]

'kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [116]:
def pred_answer(usr_query):
    df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
    df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
    usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)
    idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()
    return df['A'][idx]

In [117]:
while True:
    usr_query = input("Ask a query: ")
    if usr_query == "exit":
        break
    else:
        print(pred_answer(usr_query))
        print("-----------------------------------------------")

Ask a query: tell me about kandi
kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.
-----------------------------------------------
Ask a query: What components does kandi cover
kandi helps you select software components across:
Packages from all package managers and repositories
Source Code across all major code repositories
Cloud Functions and APIs across all hyperscale cloud providers
-----------------------------------------------
Ask a query: exit
